# Preliminaries

In [2]:
import torch
from torch import nn
from torch import optim
from torch.utils import data
import wandb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from keras import layers
from sklearn.model_selection import train_test_split
from keras.layers import Embedding, Dense, LSTM, Dropout
from keras.losses import BinaryCrossentropy
from keras.models import Sequential
from keras.optimizers import adam_v2
from keras.preprocessing.sequence import pad_sequences

In [3]:
config = dict()

config['path_workspace'] = 'C:\\Users\\SafontAndreu\\Workspace\\Visual Studio Code\\ps_hiv\\'
config['path_inputs'] = 'C:\\Users\\SafontAndreu\\'

config['path_database'] = config.get('path_workspace') + 'data\\'
config['path_models'] = config.get('path_inputs') + 'Models\\'
config['path_results'] = config.get('path_inputs') + 'Results\\'
config['path_ontology'] = config.get('path_inputs') + 'Ontology\\'

# Data

In [4]:
#X_name = 'PR Seq'
#X_name = 'RT Seq'
#X_name = 'Seq' # concatenate sequences
#X_name = 'Count' # concatenate counts
X_name = 'All' # concatenate viral/ct load with sequences

y_name = 'Resp' # binary prognosis

In [5]:
train_data = df = pd.read_csv(config.get('path_database')+'training_data.csv', header=0, index_col=False, encoding='utf-8', low_memory=False)
test_data = df = pd.read_csv(config.get('path_database')+'test_data_mod.csv', header=0, index_col=False, encoding='utf-8', low_memory=False)
sol_data = df = pd.read_csv(config.get('path_database')+'hivprogression_solution.csv', header=0, index_col=False, encoding='utf-8', low_memory=False)
train_data.head()

,PatientID,Resp,PR Seq,RT Seq,VL-t0,CD4-t0
0,1,0,CCTCAAATCACTCTTTGGCAACGACCCCTCGTCCCAATAAGGATAG...,CCCATTAGTCCTATTGAAACTGTACCAGTAAAGCTAAAGCCAGGAA...,4.3,145
1,2,0,CCTCAAATCACTCTTTGGCAACGACCCCTCGTCGCAATAAAGATAG...,CCCATTAGTCCTATTGAAACTGTACCAGTAAAATTAAAGCCAGGAA...,3.6,224
2,3,0,CCTCAAATCACTCTTTGGCAACGACCCCTCGTCGCAATAAAGGTAG...,CCCATTAGTCCTATTGAAACTGTACCAGTAAAATTAAAGCCAGGAA...,3.2,1017
3,4,0,CCTCAAATCACTCTTTGGCAACGACCCCTCGTCGCAATAAGGATAG...,CCCATTAGTCCTATTGAAACTGTACCAGTAAAATTAAAGCCAGGAA...,5.7,206
4,5,0,CCTCAAATCACTCTTTGGCAACGACCCCTCGTCGCAGTAAAGATAG...,CCCATTAGTCCTATTGAAACTGTACCAGTAAAATTAAAGCCAGGAA...,3.5,572


## Tokenize and vocab

In [6]:
import collections

def tokenize(seqs):
    return [tokenize_line(seq) for seq in seqs]

def tokenize_line(seq):
    if not pd.isna(seq):
        return list(seq)
    return []

class Vocab:
    def __init__(self, tokens):
        counter = count_corpus(tokens)
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1],
                                   reverse=True)
        self.idx_to_token = ['<unk>']
        self.token_to_idx = {token: idx
                             for idx, token in enumerate(self.idx_to_token)}
        for token, freq in self._token_freqs:
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1

    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

    @property
    def unk(self): 
        return 0

    @property
    def token_freqs(self):
        return self._token_freqs

def count_corpus(tokens):
    if len(tokens) == 0 or isinstance(tokens[0], list):
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)

## Data preprocessing (training)

In [11]:
seq_pr = train_data['PR Seq'].values.tolist()
seq_pr = [b for b in seq_pr if isinstance(b, str)]

seq_pr_unique = ''
for ele in seq_pr:
    if isinstance(ele, str):
        seq_pr_unique += ''.join(set(ele))
seq_pr_unique = ''.join(set(seq_pr_unique))
seq_pr_unique = ''.join(sorted(seq_pr_unique))
print('Unique nucleotides in PR Sequence = ', seq_pr_unique)
print('Min. = ', len(min(seq_pr, key=len)))
print('Max. = ', len(max(seq_pr, key=len)))



seq_rt = train_data['RT Seq'].values.tolist()
seq_rt = [b for b in seq_rt if isinstance(b, str)]

seq_rt_unique = ''
for ele in seq_rt:
    if isinstance(ele, str):
        seq_rt_unique += ''.join(set(ele))
seq_rt_unique = ''.join(set(seq_rt_unique))
seq_rt_unique = ''.join(sorted(seq_rt_unique))
print('Unique nucleotides in RT Sequence = ', seq_rt_unique)
print('Min. = ', len(min(seq_rt, key=len)))
print('Max. = ', len(max(seq_rt, key=len)))

Unique nucleotides in PR Sequence =  ABCDGHKMNRSTVWY
Min. =  216
Max. =  297
Unique nucleotides in RT Sequence =  ABCDGHKMNRSTVWY
Min. =  579
Max. =  1482


In [7]:
all_features = train_data.iloc[:, 2:]
print(all_features.shape)
# one can assume if Seqs are not present it is a bad sign for survival
all_features["PR SeqNan"] = all_features["PR Seq"].apply(lambda x: pd.isna(x)).astype(bool)
all_features["RT SeqNan"] = all_features["RT Seq"].apply(lambda x: pd.isna(x)).astype(bool)
numeric_features = all_features.dtypes[(all_features.dtypes != 'object') & (all_features.dtypes != 'bool')].index
mean_numerical_features = all_features[numeric_features].mean()
std_numerical_features = all_features[numeric_features].std()
all_features[numeric_features] = all_features[numeric_features].apply(lambda x: (x - x.mean()) / x.std() + 1e-4)
vt_mean = all_features["VL-t0"].mean()
cd4_mean = all_features["CD4-t0"].mean()
all_features["VL-t0"] = all_features["VL-t0"].fillna(vt_mean)
all_features["CD4-t0"] = all_features["CD4-t0"].fillna(cd4_mean)
all_features.head()

# Add results
all_features[y_name] = train_data[y_name]
print(all_features.shape)

(1000, 4)
(1000, 7)


### Subsets (training)

#### Select input

In [8]:
# PR Seq
tokens_pr = tokenize(all_features["PR Seq"].values)
vocab_pr = Vocab(tokens_pr)
list(vocab_pr.token_to_idx.items())
all_features["PR Seq"] = all_features["PR Seq"].apply(lambda x: vocab_pr[tokenize_line(x)])
#all_features["PR Seq"]

# RT Seq
tokens_rt = tokenize(all_features["RT Seq"].values)
vocab_rt = Vocab(tokens_rt)
list(vocab_rt.token_to_idx.items())
all_features["RT Seq"] = all_features["RT Seq"].apply(lambda x: vocab_rt[tokenize_line(x)])
#all_features["RT Seq"]


# RT Seq
all_features["Seq"] = all_features["PR Seq"] + all_features["RT Seq"]


# All
a_ = all_features["VL-t0"]
b_ = all_features["CD4-t0"]
c_ = all_features["Seq"]
matrix = []
for i in range(len(a_)):
    row = [a_[i]] + [b_[i]] + c_[i]
    matrix.append(row)
all_features['All'] = matrix

In [24]:
X = all_features[X_name].values
y = all_features[y_name].values

x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=42)

print(x_train.shape)
print(y_train.shape)
print(x_valid.shape)
print(y_valid.shape)

(750,)
(750,)
(250,)
(250,)


## Data preprocessing (test)

In [11]:
all_features_test = test_data.iloc[:, 2:]
print(all_features_test.shape)
# one can assume if Seqs are not present it is a bad sign for survival
all_features_test["PR SeqNan"] = all_features_test["PR Seq"].apply(lambda x: pd.isna(x)).astype(bool)
all_features_test["RT SeqNan"] = all_features_test["RT Seq"].apply(lambda x: pd.isna(x)).astype(bool)
numeric_features_test = all_features_test.dtypes[(all_features_test.dtypes != 'object') & (all_features_test.dtypes != 'bool')].index
mean_numerical_features_test = all_features_test[numeric_features_test].mean()
std_numerical_features_test = all_features_test[numeric_features_test].std()
all_features_test[numeric_features_test] = all_features_test[numeric_features_test].apply(lambda x: (x - x.mean()) / x.std() + 1e-4)
vt_mean_test = all_features_test["VL-t0"].mean()
cd4_mean_test = all_features_test["CD4-t0"].mean()
all_features_test["VL-t0"] = all_features_test["VL-t0"].fillna(vt_mean_test)
all_features_test["CD4-t0"] = all_features_test["CD4-t0"].fillna(cd4_mean_test)
all_features_test.head()

# Add results
all_features_test[y_name] = test_data[y_name]
print(all_features_test.shape)

(692, 4)
(692, 7)


### Subsets (test)

#### Select input

In [29]:
# PR Seq
tokens_pr_test = tokenize(all_features_test["PR Seq"].values)
vocab_pr_test = Vocab(tokens_pr_test)
list(vocab_pr_test.token_to_idx.items())
all_features_test["PR Seq"] = all_features_test["PR Seq"].apply(lambda x: vocab_pr_test[tokenize_line(x)])
#all_features["PR Seq"]

# RT Seq
tokens_rt_test = tokenize(all_features_test["RT Seq"].values)
vocab_rt_test = Vocab(tokens_rt_test)
list(vocab_rt_test.token_to_idx.items())
all_features_test["RT Seq"] = all_features_test["RT Seq"].apply(lambda x: vocab_rt_test[tokenize_line(x)])
#all_features["RT Seq"]


# RT Seq
all_features_test["Seq"] = all_features_test["PR Seq"] + all_features_test["RT Seq"]


# All
a_ = all_features_test["VL-t0"]
b_ = all_features_test["CD4-t0"]
c_ = all_features_test["Seq"]
matrix_test = []
for i in range(len(a_)):
    row = [a_[i]] + [b_[i]] + c_[i]
    matrix_test.append(row)
all_features_test['All'] = matrix_test

In [30]:
x_test = all_features_test[X_name]
y_test = all_features_test[y_name]

# Training

## Parameters

In [32]:
class_weight = {0: 0.2, # no improvement (80%)
                1: 0.8} # improvement (20%)

additional_metrics = ['accuracy']
batch_size = 8
embedding_output_dims = 8
loss_function = BinaryCrossentropy()
max_sequence_length = max(all_features[X_name].apply(len))
print('Max. seq length = ', max_sequence_length)
num_distinct_words = len(vocab_pr)+1
print('Size of vocabular = ', num_distinct_words)
epochs = 50
lr = 2e-3
optimizer = adam_v2.Adam(learning_rate=lr, decay=lr/epochs)
verbosity_mode = 1

Max. seq length =  1781
Size of vocabular =  17


In [33]:
# Disable eager execution
#tf.compat.v1.disable_eager_execution()

# Pad all sequences
padded_x_train = pad_sequences(x_train, maxlen=max_sequence_length, value = 0.0) # 0.0 because it corresponds with <PAD>
padded_x_valid = pad_sequences(x_valid, maxlen=max_sequence_length, value = 0.0) # 0.0 because it corresponds with <PAD>

## Model

In [34]:
model = Sequential()

if X_name == 'PR Seq' or X_name == 'RT Seq': # LSTM
    model.add(Embedding(num_distinct_words, embedding_output_dims, input_length=max_sequence_length))
    model.add(LSTM(60))
    model.add(Dense(max_sequence_length/2, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(16, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

else: # NN
    model.add(Dense(128, input_dim=max_sequence_length, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(16, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))



# Compile the model
model.compile(optimizer=optimizer, loss=loss_function, metrics=additional_metrics)

# Give a summary
#model.build((len(X[0]), 1)) # `input_shape` is the shape of the input data
                         # e.g. input_shape = (None, 32, 32, 3)
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 128)               228096    
                                                                 
 dropout_15 (Dropout)        (None, 128)               0         
                                                                 
 dense_22 (Dense)            (None, 64)                8256      
                                                                 
 dropout_16 (Dropout)        (None, 64)                0         
                                                                 
 dense_23 (Dense)            (None, 16)                1040      
                                                                 
 dropout_17 (Dropout)        (None, 16)                0         
                                                                 
 dense_24 (Dense)            (None, 1)                

In [35]:
# Train the model
history = model.fit(padded_x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=verbosity_mode, validation_data=(padded_x_valid, y_valid), class_weight=class_weight)

Epoch 1/50
94/94 [==============================] - 1s 6ms/step - loss: 1.1765 - accuracy: 0.5387 - val_loss: 0.7241 - val_accuracy: 0.2160
Epoch 2/50
94/94 [==============================] - 0s 4ms/step - loss: 0.2310 - accuracy: 0.2293 - val_loss: 0.7086 - val_accuracy: 0.2160
Epoch 3/50
94/94 [==============================] - 0s 4ms/step - loss: 0.2228 - accuracy: 0.2027 - val_loss: 0.7063 - val_accuracy: 0.2160
Epoch 4/50
94/94 [==============================] - 0s 5ms/step - loss: 0.2234 - accuracy: 0.2027 - val_loss: 0.7045 - val_accuracy: 0.2160
Epoch 5/50
94/94 [==============================] - 0s 5ms/step - loss: 0.2236 - accuracy: 0.2013 - val_loss: 0.7058 - val_accuracy: 0.2160
Epoch 6/50
94/94 [==============================] - 0s 5ms/step - loss: 0.2230 - accuracy: 0.2027 - val_loss: 0.7022 - val_accuracy: 0.2160
Epoch 7/50
94/94 [==============================] - 0s 5ms/step - loss: 0.2230 - accuracy: 0.2027 - val_loss: 0.6990 - val_accuracy: 0.2280
Epoch 8/50
94/94 [==

# Evaluation

In [36]:
padded_inputs_test = pad_sequences(x_test, maxlen=max_sequence_length, value = 0.0) # 0.0 because it corresponds with <PAD>

test_results = model.evaluate(padded_inputs_test, y_test, verbose=False)
print(f'Test results - Loss: {test_results[0]} - Accuracy: {100*test_results[1]}%')

Test results - Loss: 0.6931788325309753 - Accuracy: 50.0%
